In [1]:
#Import the libraries
from utils import process_tweet, lookup
import pdb
from nltk.corpus import stopwords, twitter_samples
import numpy as np
import pandas as pd
import nltk
import string
from nltk.tokenize import TweetTokenizer
from os import getcwd

In [2]:
# add folder, tmp2, from our local workspace containing pre-downloaded corpora files to nltk's data path
filePath = f"{getcwd()}/../tmp2/"
nltk.data.path.append(filePath)

In [3]:
#Get the positive and negative tweets
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')

print(len(positive_tweets))
print(len(negative_tweets))

5000
5000


In [4]:
#Split the data into training and testing
pos_train = positive_tweets[:4000]
pos_test = positive_tweets[4000:]

neg_train = negative_tweets[:4000]
neg_test = negative_tweets[4000:]

In [5]:
#Merge the training and testing
X_train = pos_train + neg_train
X_test = pos_test + neg_test

In [6]:
#Create the labels for training and testing data
y_train = np.append(np.ones(len(pos_train)), np.zeros(len(neg_train)))
y_test = np.append(np.ones(len(pos_test)), np.zeros(len(neg_test)))

# Process the data

In [7]:
#Use process tweets function to remove stopword, symbols, punctuation marks and finally stem the word
custom_tweet = "RT @samip @xyz Hi there!! how are you?? Have a good day :) Enjoy and learn NLP. #good #morning http://www.goodmoring.com"
processed_tweet = process_tweet(custom_tweet)
print(processed_tweet)

['hi', 'good', 'day', ':)', 'enjoy', 'learn', 'nlp', 'good', 'morn']


In [8]:
#Create a function to create the dictionary with word and label as key and number of time word appeared in that label as value
def count_tweets(result, tweets, ys):
    
    #Iterate to list of tweets and ots label
    for y, tweet in zip(ys, tweets):
        
        #Get all words of processed tweet
        for word in process_tweet(tweet):
            #Create a pair
            pair = (word, y)
            
            #Get the occurence of pair as value
            if pair in result:
                result[pair] += 1
            else:
                result[pair] = 1
    return result

In [9]:
#Test the function
result = {}
tweets = ['i am happy', 'i am tricked', 'i am sad', 'i am tired', 'i am tired']
ys = [1, 0, 0, 0, 0]
count_tweets(result, tweets, ys)

{('happi', 1): 1, ('trick', 0): 1, ('sad', 0): 1, ('tire', 0): 2}

# Train the Naive Bayes Model

In [10]:
#Create a frequency dictionary for training data
freqs = count_tweets({}, X_train, y_train)

In [11]:
#Train the Naive Bayes using series of steps
def train_naive_bayes(freqs, X_train, y_train):
    
    loglikelihood = {}
    logprior = 0
    
    #Calculate V- the number of unique words in the vocabulory
    vocab = set([pair[0] for pair in freqs.keys()])
    V = len(vocab)
    
    #Calculate the total number of words in positive and negative tweets
    N_pos, V_pos, N_neg, V_neg = 0, 0, 0, 0
    
    #Iterate through all pairs
    for pair in freqs.keys():
        #Check for positive
        if pair[1] > 0:
            V_pos += 1
            N_pos += freqs[pair]
        
        else:
            V_neg += 1
            N_neg += freqs[pair]
    
    #Calculate the number of tweets
    D = len(y_train)
    
    #Calculate the number of positive and negative tweets
    D_pos = len(list(filter(lambda x: x > 0, y_train)))
    D_neg = len(list(filter(lambda x: x <= 0, y_train)))
    
    #Calculate the logprior
    logprior = np.log(D_pos) - np.log(D_neg)
    
    #Calculate the total loglikelihood
    for word in vocab:
        
        #Get the positive and negative frequency
        freq_pos = lookup(freqs, word, 1)
        freq_neg = lookup(freqs, word, 0)
        
        #Calculate the probability of each word
        p_w_pos = (freq_pos + 1) / (N_pos + V)
        p_w_neg = (freq_neg + 1) / (N_neg + V)
        
        loglikelihood[word] = np.log(p_w_pos) - np.log(p_w_neg)
        
    return logprior, loglikelihood
    

In [12]:
#Get the actual value of logprior and loglikelihood for our training data
logprior, loglikelihood = train_naive_bayes(freqs, X_train, y_train)
print(logprior)
print(len(loglikelihood))

0.0
9089


# Test the Naive Bayes

In [13]:
#Function to create the prediction of final sentiment
def naive_bayes_predict(tweet, logprior, loglikelihood):
    
    #Get the processed tweet
    word_list = process_tweet(tweet)
    
    #Initialize the probability to zero
    p = 0
    
    #Add the logprior
    p += logprior
    
    #Iterate over the processed tweet and add the likelihood for each word (0 if not in likelihood dictionary)
    for word in word_list:
        
        if word in loglikelihood:
            p += loglikelihood[word]
        
    return p

In [14]:
#Test on a sample tweet
custom_tweet = "He smiled."
p = naive_bayes_predict(custom_tweet, logprior, loglikelihood)
print(p)

1.5740278623499178


In [15]:
#Implement test naive bayes to go through all prediction of test set
def naive_bayes_test(X_test, y_test, logprior, loglikelihood):
    
    #Create an empty list to store predictions
    y_hat = []
    
    #Iterate through all tweets
    for tweet in X_test:
        #Check if overall sentiment is greater than zero, set positive (0) else negative (1)
        if naive_bayes_predict(tweet, logprior, loglikelihood) > 0:
            y_hat.append(1)
        else:
            y_hat.append(0)
    
    #Get the mean error
    error = np.mean(np.absolute(y_hat - y_test))
    
    #Accuracy is 1 - error
    accuracy = 1 - error
    
    return accuracy

In [16]:
#Get the accuracy for our data
accuracy = naive_bayes_test(X_test, y_test, logprior, loglikelihood)
print("Accuracy of Naive Bayes is: %0.4f" %accuracy)

Accuracy of Naive Bayes is: 0.9940


In [17]:
#Check the output sentiment for some custom tweets
for tweet in ['I am happy', 'I am bad', 'this movie should have been great.', 'great', 'great great', 'great great great', 'great great great great']:
    # print( '%s -> %f' % (tweet, naive_bayes_predict(tweet, logprior, loglikelihood)))
    p = naive_bayes_predict(tweet, logprior, loglikelihood)
#     print(f'{tweet} -> {p:.2f} ({p_category})')
    print(f'{tweet} -> {p:.2f}')

I am happy -> 2.15
I am bad -> -1.29
this movie should have been great. -> 2.14
great -> 2.14
great great -> 4.28
great great great -> 6.41
great great great great -> 8.55


# Filter the words by ratio of Positive to Negative

In [18]:
def get_ratio(freqs, word):
    #Set the empty dictionary
    pos_neg_ratio = {'positive' : 0, 'negative': 0, 'ratio' : 0.0}
    
    #Get the positive word count
    pos_neg_ratio['positive'] = lookup(freqs, word, 1)
    
    #Get the negative word count
    pos_neg_ratio['negative'] = lookup(freqs, word, 0)
    
    #Calculate the ratio by adding 1 at both side (prevent division by zero)
    pos_neg_ratio['ratio'] = (pos_neg_ratio['positive'] + 1) / (pos_neg_ratio['negative'] + 1)
    
    return pos_neg_ratio

In [19]:
#Check on some words (Give the stem word as input)
get_ratio(freqs, 'good')

{'positive': 191, 'negative': 83, 'ratio': 2.2857142857142856}

In [20]:
def get_words_by_threshold(freqs, label, threshold):
    '''
    Input:
        freqs: dictionary of words
        label: 1 for positive, 0 for negative
        threshold: ratio that will be used as the cutoff for including a word in the returned dictionary
    Output:
        word_set: dictionary containing the word and information on its positive count, negative count, and ratio of positive to negative counts.
        example of a key value pair:
        {'happi':
            {'positive': 10, 'negative': 20, 'ratio': 0.5}
        }
    '''
    word_list = {}

    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    for key in freqs.keys():
        word, _ = key

        # get the positive/negative ratio for a word
        pos_neg_ratio = get_ratio(freqs, word)

        # if the label is 1 and the ratio is greater than or equal to the threshold...
        if label == 1 and pos_neg_ratio['ratio'] >= threshold:

            # Add the pos_neg_ratio to the dictionary
            word_list[word] = pos_neg_ratio

        # If the label is 0 and the pos_neg_ratio is less than or equal to the threshold...
        if label == 0 and pos_neg_ratio['ratio'] <= threshold:

            # Add the pos_neg_ratio to the dictionary
            word_list[word] = pos_neg_ratio

        # otherwise, do not include this word in the list (do nothing)

    ### END CODE HERE ###
    return word_list


In [21]:
#Check for negative words
word_list = get_words_by_threshold(freqs, 0, 0.05)
word_list

{':(': {'positive': 1, 'negative': 3663, 'ratio': 0.0005458515283842794},
 ':-(': {'positive': 0, 'negative': 378, 'ratio': 0.002638522427440633},
 'zayniscomingbackonjuli': {'positive': 0, 'negative': 19, 'ratio': 0.05},
 '26': {'positive': 0, 'negative': 20, 'ratio': 0.047619047619047616},
 '>:(': {'positive': 0, 'negative': 43, 'ratio': 0.022727272727272728},
 'lost': {'positive': 0, 'negative': 19, 'ratio': 0.05},
 '♛': {'positive': 0, 'negative': 210, 'ratio': 0.004739336492890996},
 '》': {'positive': 0, 'negative': 210, 'ratio': 0.004739336492890996},
 'beli̇ev': {'positive': 0, 'negative': 35, 'ratio': 0.027777777777777776},
 'wi̇ll': {'positive': 0, 'negative': 35, 'ratio': 0.027777777777777776},
 'justi̇n': {'positive': 0, 'negative': 35, 'ratio': 0.027777777777777776},
 'ｓｅｅ': {'positive': 0, 'negative': 35, 'ratio': 0.027777777777777776},
 'ｍｅ': {'positive': 0, 'negative': 35, 'ratio': 0.027777777777777776}}

In [22]:
#Check for positive words
get_words_by_threshold(freqs, label=1, threshold=10)

{'followfriday': {'positive': 23, 'negative': 0, 'ratio': 24.0},
 'commun': {'positive': 27, 'negative': 1, 'ratio': 14.0},
 ':)': {'positive': 2847, 'negative': 2, 'ratio': 949.3333333333334},
 'flipkartfashionfriday': {'positive': 16, 'negative': 0, 'ratio': 17.0},
 ':D': {'positive': 498, 'negative': 0, 'ratio': 499.0},
 ':p': {'positive': 103, 'negative': 0, 'ratio': 104.0},
 'influenc': {'positive': 16, 'negative': 0, 'ratio': 17.0},
 ':-)': {'positive': 543, 'negative': 0, 'ratio': 544.0},
 "here'": {'positive': 20, 'negative': 0, 'ratio': 21.0},
 'youth': {'positive': 14, 'negative': 0, 'ratio': 15.0},
 'bam': {'positive': 44, 'negative': 0, 'ratio': 45.0},
 'warsaw': {'positive': 44, 'negative': 0, 'ratio': 45.0},
 'shout': {'positive': 11, 'negative': 0, 'ratio': 12.0},
 ';)': {'positive': 22, 'negative': 0, 'ratio': 23.0},
 'stat': {'positive': 51, 'negative': 0, 'ratio': 52.0},
 'arriv': {'positive': 57, 'negative': 4, 'ratio': 11.6},
 'via': {'positive': 60, 'negative': 1, 

# Error Analysis

In [23]:
#Print some of the misclassified tweet
print("Truth Predicted Tweet")

for x, y in zip(X_test, y_test):
    y_hat = naive_bayes_predict(x, logprior, loglikelihood)
    if y_hat != (np.sign(y_hat) > 0):
        print('%d\t%0.2f\t%s' % (y, np.sign(y_hat) > 0, ' '.join(
            process_tweet(x)).encode('ascii', 'ignore')))

Truth Predicted Tweet
1	1.00	b'bro u wan cut hair anot ur hair long liao bo sinc ord liao take easi lor treat save leav longer :) bro lol sibei xialan'
1	1.00	b"back thnx god i'm happi :)"
1	1.00	b'thought ear malfunct thank good clear one apolog :-)'
1	1.00	b'stuck centr right clown right joker left ... :)'
1	1.00	b'happi friday :-)'
1	1.00	b'follow :) x'
1	1.00	b'teenchoic choiceinternationalartist superjunior fight oppa :D'
1	1.00	b"birthday today birthday wish hope there' good news ben soon :-)"
1	1.00	b'good morn :-) friday  plan day current play shop ...'
1	1.00	b'happi friday :)'
1	1.00	b'3 good nigth :) estoy escuchando enemi god'
1	1.00	b'actual bye bye inde go take drama elsewher :)'
1	1.00	b'ff readi weekend :) smile'
1	1.00	b'work :)'
1	1.00	b':) mood bipolar'
1	1.00	b'thank :)'
1	1.00	b'hahahahahahahahahahahahahaha die :) liter front like'
1	1.00	b'yoohoo shatter record bajrangibhaijaanstorm superhappi :D'
1	1.00	b"who' awak :-)"
1	1.00	b'happi friday :-)'
1	1.00	b'pleasur

1	1.00	b'ha ha ok honey go ahead fyi still dig ya enjoy vote watch side :)'
1	1.00	b'lucki :)'
1	1.00	b'learn new skill take time perfect master'
1	1.00	b"       i'v talk u coupl time u seem nice funni :)"
1	1.00	b'agre suspici mind :)'
1	1.00	b'good afternoon :)'
1	1.00	b'whoop whoop bring fun :)'
1	0.00	b'new report talk burn calori cold work harder warm feel better weather :p'
1	1.00	b"say someth i'm give :) i'm sorri get :D"
1	1.00	b'good night :)'
1	1.00	b"there' gonna anoth one final :)"
1	1.00	b'welcom jessica enjoy :)'
1	1.00	b'best friend carina one san francisco let know want get touch :)'
1	1.00	b'thank mommi teret :) :) potassium rehydr drinkitallup thirstquench'
1	1.00	b"i'm sure tapir calf one like adventur mealtim :)"
1	1.00	b'uhc box larg scale one network server :p'
1	1.00	b"ever areal we'll lunch dinner :)"
1	1.00	b"here' one extremesport fan among love quadbik get touch :) bloggersrequir bloggersw"
1	1.00	b'thank ... love cake :) bear hug end'
1	1.00	b'masa nowaday b

1	1.00	b"yeah left one supposedli hurt bc right heart appar it'll fine good luck :)"
1	1.00	b'hello :) get youth job opportun follow'
1	1.00	b'ahh thank u :D'
1	1.00	b"niall follow fan :) i'm still without follow "
1	1.00	b'sister best best pre bday celebr :)'
1	1.00	b'among time fav tweet :D'
1	1.00	b'happi birthday g best life :)'
1	1.00	b'need stress time :)'
1	1.00	b'immedi plan hope someday futur :)'
1	1.00	b"could make foxy' song version instrument form sound epic beat alon :)"
1	1.00	b'second well done :)'
1	1.00	b'think hard bed :p'
1	1.00	b'tomorrow goldcoast :)'
1	1.00	b'like way said gal lelomustfal beautiful way :)'
1	1.00	b'happi save day :) hope enjoy meal sam'
1	1.00	b'thank min :)'
1	1.00	b'5g liker :) like fast'
1	1.00	b"like morn got dress bought :) i'm total love  newdress happi "
1	1.00	b'ff could anyon resist beauti smile :-) quacketyquack'
1	1.00	b'u mean hide fot :p'
1	1.00	b'aww thankyou :)'
1	1.00	b"ok good night wish troy ugli met today :) ): ok today fun i'm 

1	1.00	b"oh gina busi :-) can't wait see hear xxx soproud"
1	1.00	b'happi friday everyon hope fantast week :) friday'
1	1.00	b'ye one team :)'
1	1.00	b'happi belat birthday sweeti :)'
1	1.00	b'morn emma :)'
1	1.00	b'thank guy :)'
1	1.00	b'would say :-)'
1	1.00	b'love make two heart one :)'
1	1.00	b'thank ad us list :) make sure keep touch news light bulb made corn wast'
1	1.00	b'thank mr bee xxx :-)'
1	1.00	b'thankyou :)'
1	1.00	b'close enought :)'
1	1.00	b'good luck ... anoth potenti favourit water hole :-)'
1	1.00	b'bad boy :) burger melbourneburg'
1	1.00	b'want ft arianna :-)'
1	1.00	b'everyon follow :)'
1	1.00	b'hey lesley sorri get shall send post tonight :)'
1	1.00	b'thank esai :-)'
1	1.00	b"thank text back :) i'm text tomorrow :)"
1	1.00	b'unfollow back :)'
1	1.00	b'hope enjoy stay rotterdam know cheer jordi clasi :)'
1	1.00	b'hi hot girl say hot horni darl xx :) '
1	1.00	b'sure done :)'
1	1.00	b'salon bleach hair olaplex damag like :)'
1	1.00	b'teamwork right :D zitecofficestor

1	1.00	b'comput sex count :) funni vintag'
1	1.00	b"use i'll pick next time i'm :) yeah i'll cook nice big kitchen  "
1	1.00	b'love great collag :)'
1	1.00	b'       thank alway put smile face mind follow :) nice day x 1710'
1	1.00	b'look fun snapchat gooffeanotter snapchat kiksex snapm lesbian instagram mpoint mugshot :)'
1	1.00	b"can't sleep much want love bug made 33 token never mind :)"
1	1.00	b'gamer follow train follow retweet follow retweet gain activ follow :) 110'
1	1.00	b'look forward hear thought :) maritimen'
1	1.00	b'stat day arriv 2 new follow unfollow :) via'
1	1.00	b"there' stun show garden rh tatton park show wonder design construct plant :)"
1	1.00	b'behav els u jump_julia malema :D'
1	1.00	b'hello :) get youth job opportun follow'
1	1.00	b'great friday fren :-)'
1	1.00	b'nuf teas alreadi show us alien everyday seem get closer see alien cousin :-)'
1	1.00	b'new monitor good :D'
1	1.00	b'good morn kimmi :)'
1	1.00	b"channel' show planetbollywoodnew full epi avail onlin 

0	0.00	b'much miss :('
0	0.00	b'nawe :('
0	0.00	b'want white french bulldog :('
0	0.00	b'wanna feel love :('
0	0.00	b"aww poor wish help even though can't realli help much :("
0	0.00	b"i'm sorri see tweet :( 2 point x"
0	0.00	b'miss macho man :('
0	0.00	b'hav puriti back :('
0	0.00	b'kwento :( fbc'
0	0.00	b'3 day without talk bae :('
0	0.00	b'whaaat nooo :('
0	0.00	b"pleas tweet someth i'm sad :( cheer pleas .."
0	0.00	b'miss brown hair :('
0	0.00	b'nakakapikon yung nagbabasa ka ng blog comment info full peopl ask damn question answer post :('
0	0.00	b'lead caus cancer children five :('
0	0.00	b"alon :-( :'( :-\\"
0	1.00	b'hello info possibl interest jonatha close join beti :( great'
0	0.00	b"i'm sorri peopl shit respect person space :("
0	0.00	b'nooo :( sogok ice cream'
0	0.00	b'thank send premium write instrument howev dastardli swine stole envelop :('
0	0.00	b'im get sick tire pipol say im short ... short ... tini :('
0	0.00	b'fair enough :( would anyon seem tad fuck  .. wiper wiper

0	0.00	b'lot peopl suggest still :('
0	0.00	b'follback pl :('
0	0.00	b'get pace maker oh :('
0	0.00	b'ah minc :('
0	0.00	b'molli deserv much higher place :('
0	0.00	b"i'm hungri :("
0	0.00	b'wanna good time one hit date much ask :('
0	0.00	b'feel sucki w run nose cough sore throat fever :-('
0	0.00	b'hate hot weather :( graduat ceremoni later boy gonna wear suit everyon go die'
0	0.00	b"let' go parti ... pant ew :( gross"
0	0.00	b'someon niall layout give :('
0	0.00	b'hey christin moodi :('
0	0.00	b'miss :('
0	0.00	b'throwback :('
0	0.00	b'that good enough reason :( pleas dont leav your one fav barb'
0	0.00	b"cooper i'm convinc neither 2 pleas i'm concern creasi win deputi :("
0	0.00	b'tast bad though :('
0	0.00	b'joke man :('
0	0.00	b"never remov minion car 23 day run batteri banana' :("
0	0.00	b'want sponsor acc new even 1k :('
0	0.00	b'told u guy bday today :( gift want paypal'
0	0.00	b'goodby stage alreadi :-('
0	0.00	b'shit idk time :('
0	0.00	b'today tire :-('
0	0.00	b'pretti :('

0	0.00	b"joe i'm sick come round make soup :("
0	0.00	b'cri :( pdapaghimok'
0	0.00	b"stori life :( haha look who' talk"
0	0.00	b'work :( smh'
0	0.00	b'anyon awak :-('
0	0.00	b'ya sorri :-('
0	0.00	b'friend :('
0	0.00	b'use flexibl :('
0	0.00	b"taxi driver go realli fast i'm scare :("
0	0.00	b'bet wonder happen :('
0	0.00	b'sheet hurt nanaman :('
0	0.00	b'need small ladi walk back sore ... :('
0	0.00	b'want meet :('
0	0.00	b'alex fell asleep :('
0	0.00	b'justin come scotland :('
0	0.00	b"ok i'll write u everi week :("
0	0.00	b'curv tummi big .. get flat ab :('
0	0.00	b'look like :('
0	0.00	b"feel sick stomach hurt i'v done cri night can't sleep :("
0	0.00	b'wait gone liter :('
0	0.00	b"that' quot tweet :("
0	0.00	b'cream pinay pie soup :('
0	0.00	b'one :('
0	0.00	b'jadi langsung flasback .. :( dont cri babyy ;('
0	0.00	b'okay miss :('
0	0.00	b'listen album live franc :('
0	0.00	b'time like style :| like :('
0	0.00	b'feel sick :('
0	0.00	b'account meant hit us lunch time :('
0	0.00	b'mis

0	0.00	b'happi princess today :( sigh get work done x purpl princess edit'
0	0.00	b'babee :( demn hotaisndonwyvauwjoqhsjsnaihsuswtf'
0	0.00	b'someth wrong :('
0	0.00	b'word explain way miss :('
0	0.00	b'think need 2 year train beat record sia :('
0	0.00	b'niram ya geng :( fikri anna 6 other tirtagangga hotel '
0	0.00	b'char im realli sick :( one font size 8 minion float font size 12 sick kid pl :('
0	0.00	b'bestfriend like ed plsss :('
0	0.00	b'riprishikeshwari soul rest peac :('
0	0.00	b':( unfollow like 300 peopl caus follow 1k peopl difficult'
0	0.00	b'enough time listen artist music :-('
0	0.00	b'take ice creamist pleasee :('
0	0.00	b'alway go terribl time :('
0	0.00	b':-( challeng though pleas check fb page entri rather substitut thank'
0	0.00	b"can't even see hyungwon :("
0	0.00	b'tell pl eat :-('
0	0.00	b'that life get call peopl havent seen 20 year alway favour'
0	0.00	b'omg :( everi time would complain skin mom everyon get pimpl like although'
0	0.00	b'letsfootbal atk greymind

0	0.00	b'wonho remind someon >:( cant put finger'
0	0.00	b'wont work today feel ill :( tummi bug xx'
0	0.00	b'last day beach :-( :-( :-( ...'
0	0.00	b'rain :('
0	0.00	b'visual studio 2015 got switch alreadi broke lot code :('
0	0.00	b"serious unfair ask question readi answer win :( ='( busi ask question"
0	0.00	b'mission bear fail :('
0	0.00	b':-( ugli af :-('
0	0.00	b'pap spirit anim  dont :('
0	0.00	b"total ship danzel i'm craft can't go vidcon would sell soul devil get hug dan phil :("
0	0.00	b'hate :('
0	0.00	b"he' gona miss sheff utd york probabl wait visa leav gim 1 friendli bet start season bench :("
0	0.00	b'lead us gone :('
0	0.00	b'   mess :('
0	0.00	b'ironi harm .. :('
0	0.00	b'miss much :( xxx'
0	0.00	b'motiv :('
0	0.00	b'point yolo sort die lactos ... ugh like 3 month pregnant bloat :('
0	0.00	b'realli sorri cant find imag mention :('
0	0.00	b'im get olli tweet notif :('
0	0.00	b'go home tomorrow :-('
0	0.00	b"bad i'm poor :("
0	0.00	b'team kill :('
0	0.00	b'way :('
0	0.00

In [24]:
#Predict for the custom tweet
custom_tweet = "I am learning as I like NLP :("
p = naive_bayes_predict(custom_tweet, logprior, loglikelihood)
print(p)

-6.91035611400191
